# Hierarchical Louvain

In [18]:
import math
import networkx as nx
import igraph as ig
import leidenalg as la
from ipysigma import Sigma, SigmaGrid
from ebbe import partitioned_items
from pelote import read_graphology_json

Sigma.set_defaults(max_categorical_colors=100)

In [88]:
corpus = nx.Graph(read_graphology_json('eurosis.json'))
                 
# for node, attr in corpus.nodes.data():
#     del attr['x']
#     del attr['y']
#     del attr['color']

## Helpers

In [96]:
def semi_induced_subgraph(graph, nodes, mapping=None):
    ext = "<EXT>"

    induced = graph.subgraph(nodes).copy()

    for _, _, attr in induced.edges.data():
        attr['weight'] = 1

    for node in nodes:
        for neighbor in graph.neighbors(node):
            if neighbor in nodes:
                continue

            target = ext

            if mapping is not None:
                target = "<EXT-" + str(mapping[neighbor]) + ">"

            if induced.has_edge(node, target):
                induced[node][target]["weight"] += 1
            else:
                induced.add_edge(node, target, weight=1)

    return induced

In [108]:
_l = leiden_modularity(corpus)
_g = partitioned_items(((v, k) for k, v in _l.items()), container=set)
Sigma(
    semi_induced_subgraph(corpus, max(_g, key=len), _l),
    node_color=lambda u: "EXT" in u,
    edge_color="weight",
    edge_zindex="weight",
    edge_color_gradient=('#ccc', 'black'),
    node_size=corpus.degree
)

Sigma(nx.Graph with 206 nodes and 1,445 edges)

## Top-down approach

In [3]:
def leiden_modularity(g: nx.Graph, weighted: bool = False):
    ig_g = ig.Graph.from_networkx(g)

    weights = None

    if weighted:
        weights = [w for _, _, w in g.edges(data="weight")]
    
    partition = la.find_partition(ig_g, la.ModularityVertexPartition, weights=weights)
    return {n: m for n, m in zip(g, partition._membership)}

In [106]:
def hierarchical_leiden_modularity(g: nx.Graph, keep_ext: bool = False):
    first_level = leiden_modularity(g)
    clusters = partitioned_items(((v, k) for k, v in first_level.items()), container=set)

    threshold = math.ceil(math.log(g.order()))
    threshold = 0

    result = {}
    
    for i, cluster in enumerate(clusters):
        if len(cluster) < threshold:
            for node in cluster:
                result[node] = str(i)

            continue

        if keep_ext:
            subgraph = semi_induced_subgraph(g, cluster, first_level)
            second_level = leiden_modularity(subgraph, weighted=True)
        else:
            subgraph = g.subgraph(cluster)
            second_level = leiden_modularity(subgraph)

        for j, sub_cluster in enumerate(partitioned_items((v, k) for k, v in second_level.items())):
            if len(sub_cluster) > threshold:
                for node in sub_cluster:
                    result[node] = str(i) + '_' + str(j)
            else:
                for node in sub_cluster:
                    result[node] = str(i)

        # for node, j in second_level.items():
        #     result[node] = str(i) + '_' + str(j)

    return first_level, result

In [107]:
hl = hierarchical_leiden_modularity(corpus)
hlsi = hierarchical_leiden_modularity(corpus, True)

SigmaGrid(
    corpus,
    node_size=corpus.degree,
    views=[
        {"name": "Top level", "node_color": leiden_modularity(corpus), "default_node_border_color": "gray"},
        {"name": "Bottom level", "node_color": hl[1], "default_node_border_color": "gray"},
        {"name": "Bottom level semi-induced", "node_color": hlsi[1], "default_node_border_color": "gray"}
    ],
    columns=3
)

## Bottom-up approach

In [6]:
partitions = list(nx.community.louvain_partitions(corpus, resolution=1))
len(partitions)

3

In [7]:
SigmaGrid(corpus, node_size=corpus.degree, views=[{"name": "Level " + str(i), "node_color": p} for i, p in enumerate(partitions)])

## Comparison

In [8]:
other = read_graphology_json('NETWORK_CorpusMedia_DEFACTO_medialab_SciencesPo_V1.json')

for node, attr in other.nodes.data():
    del attr['x']
    del attr['y']
    del attr['color']

other_partitions = list(nx.community.louvain_partitions(other, resolution=1))

In [9]:
len(other_partitions)

1

In [112]:
h_other = hierarchical_leiden_modularity(other, True)

SigmaGrid(
    other,
    node_size=other.degree,
    views=[
        {"name": "Top Level", "node_color": h_other[0]},
        {"name": "Bottom Level", "node_color": h_other[1]}
    ]
)

In [110]:
SigmaGrid(
    other,
    node_size=other.degree,
    views=[
        {"name": "Top-down", "node_color": other_partitions[0]},
        {"name": "Bottom-up", "node_color": hierarchical_leiden_modularity(other)[0]}
    ]
)